# 如何建立一个深度神经网络

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
# from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 1 Initialization

对于L层的神经网络首先进行参数初始化
0层X是（12288，209） examples的数量m = 209

<table style="width:100%">


    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
   
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    
   <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    
    
   <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>

</table>

In [2]:
def initialize_parameter_deep(lay_dims):
    parameters = {}
    L = len(layer_dims)            # L是的神经网络的层数，包括0层

    #对第1到最后一层的参数初始化
    for l in range(1, L):        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        # W[l]是(n[l], n[l-1])矩阵
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        # b[l]是(n[l], 1)矩阵
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
       
    return parameters  # 把所有W和b都放到一个dictionary里储存起来

# 2 前向传播法
## 2.1 线性前向传播

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

where $A^{[0]} = X$. 

In [3]:
# 定义一个前向传播函数
def linear_forward(A, W, b):
   # 输入A[l-1], W[l], b[l]可以求出Z[l]
    Z = np.dot(W,A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b) # 把输入的A[l-1], W[l], b[l]缓存到一个tuple里面
    
    return Z, cache     

## 2.2 激活函数

**Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$.

**ReLU**: $A = RELU(Z) = max(0, Z)$.


 $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ 

In [4]:
def linear_activation_forward(A_prev, W, b, activation):
    
# 用2.1的函数算出Z[l]=W[l]A[l-1]+b[l]
    # 如果是sigmoid函数层，
    # 算出A[l]=sigmoid(Z[l])
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        # linear_cache = (A[l-1], W[l], b[l])
        A, activation_cache = sigmoid(Z)
        # activation_cache = Z[l]
    
    # 如果激励函数是reLU函数的层，A[l]=reLU(Z[l])
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))  # A是(n[l],m)矩阵
    cache = (linear_cache, activation_cache)

    return A, cache  # 输出A[l], cache = (A[l-1],W[l],b[l],Z[l])

## 2.3 L层神经网络模型

### L-1层是reLU函数层，最后一层第L层是sigmoid函数层，利用上面定义好的函数求出每一层的值。

In [5]:
# 输入参数X，和初始化的每一层的W,b
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # L层神经网络
    
    # 重复循环执行L-1遍的reLU函数层计算
    for l in range(1, L):
        A_prev = A 
        # 用2.2的函数，算出(A[l-1],W[l],b[l],Z[l])，并全部放到一个List里
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], activation='relu')
        caches.append(cache)
        '''
此循环主要是为了得到caches = [(A[0],W[1],b[1],Z[1]),
                                        .
                                        .
                                        .
                            (A[l-1],W[l],b[l],Z[l])
                                        .
                                        .
                                        .
                        (A[L-2],W[L-1],b[L-1],Z[L-1])]
        '''
    
# 上面循环最终的输出的A = A[L-1]，执行最后一层的sigmoid层计算，
# 带入linear_activation_forward函数可以得到 A[L] 和 (A[L-1],W[L],b[L],A[L])
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], activation='sigmoid')
    caches.append(cache)
    '''
    把最后一个cache加入则可以得到
    caches = [(A[0],W[1],b[1],Z[1]),
                        .
                        .
                        .
             (A[l-1],W[l],b[l],Z[l])
                        .
                        .
                        .
            (A[L-1],W[L],b[L],Z[L])]  
    ''' 
    
    assert(AL.shape == (1,X.shape[1]))      # A[L]也就是 y^是 1Xm的矩阵
            
    return AL, caches 

# 3 Cost function

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) $$

In [6]:
def compute_cost(AL, Y):
#输入由上面函数得出的最后一层输出的预测值 y^ = A[L] 和实际值 Y
    m = Y.shape[1]  
    cost = -1/m * np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    
    cost = np.squeeze(cost)     
    assert(cost.shape == ())
    
    return cost
# 输出 Cost的值

# 4 Backward propagation
## 4.1 后向传播线性部分

### $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$

### dZ[l]/dW[l] = A[l-1]
### dZ[l]/db[l] = 1
### dZ[l]/dA[l-1] = W[l]

每一层输入 $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$
输出 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$
链式法则
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

In [7]:
# 定义一个函数，输入dZ[l], cache = (A[l-1],W[l],b[l])，此可由上面2.3的函数求出。
# 输出dA[l-1], dW[l], db[l]
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True )
    dA_prev = np.dot(W.T, dZ)

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

## 4.2 后向传播线性激励函数

### 当该层是sigmoid函数层时，dZ = sigmoid_backward(dA, activation_cache)
输入（dA[l], A[l]) 得到dZ[l], 原理是$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) $$

### 当该层是relu函数层时， dZ = relu_backward(dA, activation_cache)

In [8]:
# 定义个函数，输入（dA[l], cache = (A[l-1],W[l],b[l],Z[l]), 激励函数的名字）
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
# 用4.1的函数，求出dA[l-1], dW[l], db[l]
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)    # dZ[l] = relu_backward(dA[l], A[l])
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)   # dZ[l] = sigmoid_backward(dA[l], A[l])
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db   
    # dA[l-1], dW[l], db[l] = linear_backward(dZ[l],(A[l-1],W[l],b[l]))

## 4.3 L-Model Backward
用前向传播求出的A, W, b求出梯度

In [9]:
# 定义函数用后向传播，输入A[L],Y, 和caches = (A[l-1],W[l],b[l],Z[l])的集合，求出dA,dW,db,放入dictionary中。
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # 神经网络层数
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # Y与A[L]的shape相同
    
    # 第一步先求出dA[L],对最后一层的loss function求导
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
    
    # 当前调用的缓存cache = (A[L-1],W[L],b[L],Z[L])
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    # 使用linear_activation_backward函数，可以求出 dA[L-1], dW[L], db[L]
    
    # 从L-1到1层的激励函数是reLU函数，循环计算L-1次,从l=L-2开始，到l=0为止
    for l in reversed(range(L-1)):
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        # 当前调用缓存cache = (A[l-1],W[l],b[l],Z[l])
        # 从(A[L-2],W[L-1],b[L-1],Z[L-1])到(A[0],W[1],b[1],Z[1])
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        # 用linear_activation_backward函数，计算出dA[L-2],dW[L-1],db[L-1]到dA[0],dW[1],db[1]

    return grads

## 4.4 参数更新

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} $$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} $$

In [10]:
# 输入参数W,b,dW,db,学习率α来求出经过一次梯度下降之后的W, b
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2   # 神经网络的层数

    for l in range(L):
        parameters["W" + str(l+1)] = parameters['W'+str(l+1)] - learning_rate*grads['dW'+str(l+1)]
        parameters["b" + str(l+1)] = parameters['b'+str(l+1)] - learning_rate*grads['db'+str(l+1)]
    
    return parameters

# 5 使用L层神经网络

In [11]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009

    np.random.seed(1)
    costs = []                         # 追踪cost
    
    parameters = initialize_parameters_deep(layers_dims) 
    # 初始化参数
    
    # 用梯度下降法循环迭代（这里是3000次）
    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X, parameters)  # 使用前向传播算法计算出每一层的A，W，b
       
        cost = compute_cost(AL, Y) # 计算出Cost
        
        # 使用后向传播算法计算出每一层的梯度dA[l-1],dW[l],db[l]
        grads = L_model_backward(AL, Y, caches)
       
        # 用梯度下降法更新参数W,b
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # 每100次迭代计算一下cost的值
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

# 经过N次迭代，得到优化的参数W,b

# 6 用神经网络训练好的参数预测模型

## pred_train = predict(train_x, train_y, parameters)